In [11]:
import numpy as np
import scipy.linalg as sln
import matplotlib.pyplot as plt
import warnings
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

Points
<br># -> Doc(Main)
<br>## -> My Notes
<br>### -> Missing, Corrections to do

In [64]:
def mat_col_order(a):
    m,n = a.shape
    lst = []
    for c in range(n):
        for r in range(m):
            lst.append(a[r][c])
    return lst

In [ ]:
def element_chooser(a,x):
    lst1 = mat_col_order(a)
    if max(x) <= len(lst):
        for i in x:
            lst2.append(lst1[x])
        return lst2
    else:
        return ("Out of bounds index")

Main Functions

In [ ]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [7]:
def mv(A,x,transp_flag=0):
    if np.issubdtype(A.dtype, np.number):

        ##Check this def in matlab as well
        if np.any(A):
            if transp_flag == 0:
                x = A @ x
            else:
                x = A.T @ x
    else:
        
        ##Check the logic here as well
        if transp_flag == 0:
            x = element_chooser(A,x)
            #print('Transpose flag set for function with 1 argument')
        ###Something happens here 
    return x

In [ ]:
def element(A, i=None, j=None, n=None):

    # Handling default arguments
    if i is None:
        i = []
    if j is None:
        j = []

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:
            if not i:
                e = A[:, j]
            elif not j:
                e = A[i, :]
            else:
                e = A[i, j]
        else:
            e = A[i]  # Vector
    else:  # Function
        if not j:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i:
            e = e[i]

    return e

In [ ]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

def element(A, i=None, j=None, n=None):

    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if isinstance(i, list) and isinstance(j, list):
            e = A[np.ix_(i, j)]
        else:
            e = A[i, j]
    else:  # Function
        if j is None:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i is not None:
            e = e[i]

    return e

In [43]:
def krylov_ata(A, **kwargs):

    #Unpacking kwargs to collect the parameters
    v1 = kwargs.get('v1',None)
    k = kwargs.get('k',None)
    full = kwargs.get('full',None)
    reortho = kwargs.get('reortho',None)

    #Checking is A is numeric
    if not (np.issubdtype(A.dtype, np.number)):
        return ("matrix entered is not numeric in nature")

    #Giving a value for the unpacked parameters in kwargs
    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    if k is None:
        k = 10
    if full is None:
        full = 1
    if reortho is None:
        reortho = 2

    #Preallocation of matrices Alpha and Beta
    alpha = np.zeros(k)
    if full:
        beta = np.zeros(k)
    else:
        beta = np.zeros(k-1)

    #If we want to reorthogonalise the vectors(opt 1 or 2)
    if reortho:
        V = np.zeros((len(v1),k))
        V[:,0] = v1 / np.linalg.norm(v1)
        ##Check: Made only in the python version
        U = np.zeros((A.shape[0],k))
    else:
        v = v1 / np.linalg.norm(v1) ##vector which will not be orthogonalised

    #Creating the matrices U and V using the vectors u and v
    #Will reorthogonalise if reortho = 2
    for j in range(0,k):
        if reortho:
            r = mv(A,V[:, j - 1],0)
            if (j == 0) and (reortho == 2):
                #Initialising matrix U which will be reorthogonalised
                U = np.zeros((len(r),k))
        else: 
            r = mv(A,v,0)
        if j >= 1:
            if reortho == 2:
                r = r - beta[j - 2] * U[:, j - 2]
                r = r - U[:, 0:j - 1] @ (U[:, 0:j - 1].T @ r)
            else:
                r = r - beta[j - 2] * u
        
        alpha[j - 1] = np.linalg.norm(r)
        if reortho == 2:
            U[:, j - 1] = r / alpha[j - 1]
            ##Look at how the transp_flag parameter being 1 affects the result
            r = mv(A.T, U[:, j - 1], 1)
        else:
            u = r / alpha[j - 1]
            r = mv(A.T, u, 1)
        if reortho:
            r = r - alpha[j - 1] * V[:, j - 1]
            r = r - V[:, 0:j] @ (V[:, 0:j].T @ r)
        else:
            r = r - alpha[j - 1] * v

        if (j < k):
            beta[0][j - 1] = np.linalg.norm(r)
            if reortho:
                V[:, j] = r / beta[j - 1]
            else:
                v = r / beta[j - 1]
    if not (reortho):
        V = v
    if reortho < 2:
        U = u

    #Return U, V, alpha & beta
    return V, U, alpha, beta

In [44]:
def krylov_ata_expand(A, V, U, c, k = 10):
    m = V.shape[1]
    V = np.concatenate(V, np.zeros((V.shape[0],k)))
    U = np.concatenate(U, np.zeros((U.shape[0],k)))
    alpha = np.zeros(k)
    beta = np.zeros(k)

    for j in range(m - 1, k + m):
        if j == m - 1:
            r = mv(A, V[:, j - 1], 0) - U[:, 1:j - 2] @ c
        else:
            r = mv(A, V[:, j - 1],0) - beta[j - m - 1] * U[:, j - 2]

        r = r - U[:, :j - 2] @ (U[:, :j - 2].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        U[:, j - 1] = r / alpha[j - m]
        r = mv(A.T, U[:, j - 1]) - alpha[j - m] * V[:, j - 1]
        r = r - V[:, :j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [52]:
def krylov_shur_svd(A, **kwargs):
    #Unpacking kwargs to collect the parameters
    nr = kwargs.get('nr',None)
    v1 = kwargs.get('v1',None)
    tol = kwargs.get('absrel',None)
    absrel = kwargs.get('absrel',None)
    mindim = kwargs.get('mindim',None)
    maxdim = kwargs.get('maxdim',None)
    maxit = kwargs.get('maxit',None)
    target = kwargs.get('target',None)
    info = kwargs.get('info',None)

    #Checking is A is numeric
    ###Need to fix when A is a function
    if not (np.issubdtype(A.dtype, np.number)):
        return ("matrix entered is not numeric in nature")

    #Giving a value for the unpacked parameters in kwargs
    if nr is None:
        nr = 1
    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    if tol is None:
        tol = 1e-6
    if absrel is None:
        absrel = 'rel'
    if mindim is None:
        mindim = 10
    if maxdim is None:
        maxdim = 20
    if maxit is None:
        maxit = 1000
    if target is None:
        target = np.inf
    if info is None:
        info = 0

    #Reassigning maxdim and mindim to shorter variables
    m1 = mindim
    m2 = maxdim

    #Fixing some of the parameters
    if m1 < nr:
        m1 = nr
    if m2 < m1:
        m2 = 2 * m1
    if (absrel == 'rel') and (np.issubdtype(A.dtype, np.number)):
        ##Multiplying the tol with the 1-norm of A
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((m2,m2 + 1))
    V, U, alpha, beta = krylov_ata(A, v1, m1)
    B[:m1 + 1, :m1 +1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    hist = np.zeros(maxit - 1)
    
    for k in range(maxit):
        V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:m1, m1], m2 - m1)
        ##Subtracting to match the 'm2-m1' index in MATLAB
        B[m1:, m1: m2 + 1] = np.diag(alpha) + np.diag(beta[:m2 - m1 -1], 1) 
        B[m2 - 1, m2] = beta[m2 - m1 - 1]
        X, sigma, Y = np.linalg.svd(B[:m2, :m2])
    
        ####No need to attempt the target == 0 section####
            
        
        #Restarts of Lancos algorithm
        result = element(A, list(np.arange(1, 3)), list(np.arange(2)))
        ###V = np.concatenate(((element(V[:, :m2] @ Y, 0:n, 0:m1), V[:, m2])))
        ###U = element(U[:, :m2] @ X, 0:m, 0:m1)
        ###e = element(B[:, m2].T @ X, 0:m1)
        V = np.concatenate(((element(V[:, :m2] @ Y, list(np.arange(0,n)), list(np.arange(0,m1))), V[:, m2])))
        U = element(U[:, :m2] @ X, list(np.arange(0,m)), list(np.arange(0,m1)))
        e = element(B[:, m2].T @ X, list(np.arange(0,m1)))
        ###Check the axis of concatenation
        B[:m1, :m1 + 1] = np.concatenate((np.diag(sigma[:m1]), e.T))
    
        err = np.linalg.norm(e[:nr])
        hist[k] = err
    
        if (info > 1):
            print(f'{k:4d}  {err:6.2e}')
            print(f'{sigma[:min(3,nr)]:6.2e}')
    
        if (err < tol):
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (m2 - m1) + m1
            print("Found after {k} iterations with residual = {err:6.2e}".format(k, err))
            return sigma, V, U, hist, mvs
    if info:
            print("Quit after max {k} iterations with residual = {err:6.2e}".format(k, err))
            sigma = sigma[:m1]
            V = V[:, :m1]
    return sigma, V, U, hist, mvs
            

SyntaxError: invalid syntax (358128681.py, line 67)

In [31]:
S1 = np.append(S, [0])

In [32]:
S1

array([ 16.848,  1.068,  0.000,  0.000])

In [33]:
np.diag(S1)

array([[ 16.848,  0.000,  0.000,  0.000],
       [ 0.000,  1.068,  0.000,  0.000],
       [ 0.000,  0.000,  0.000,  0.000],
       [ 0.000,  0.000,  0.000,  0.000]])

In [34]:
a = np.array([[]])

In [35]:
a

array([], shape=(1, 0), dtype=float64)

In [47]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

def element(A, i=None, j=None, n=None):

    # Handling default arguments
    if i is None:
        i = []
    if j is None:
        j = []

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:
            if not i:
                e = A[:, j]
            elif not j:
                e = A[i, :]
            else:
                e = A[i, j]
        else:
            e = A[i]  # Vector
    else:  # Function
        if not j:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i:
            e = e[i]

    return e